In [1]:
import json
import fsspec

import hist
import dask
import awkward as ak
import hist.dask as hda
import dask_awkward as dak

from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import Weights
from coffea.dataset_tools import (
    apply_to_fileset,
    max_chunks,
    preprocess,
)

import dask

In [2]:
with fsspec.open("2023_VV.json") as fin:
    wwfiles_all = json.load(fin)
    
with fsspec.open("2023_WH.json") as fin:
    whfiles_all = json.load(fin)

wwfiles = wwfiles_all["WW"]
whfiles = whfiles_all["WminusH_Hto2B_Wto2Q"] + whfiles_all["WplusH_Hto2B_Wto2Q"]

print(len(wwfiles), len(whfiles))

125 785


In [3]:
all_filesets = {}

for p in ["WW","WH"]:
    all_filesets[p] = {}
    all_filesets[p]["files"] = {}

for f in wwfiles:
    all_filesets["WW"]["files"][f] = "Events"
    
for f in whfiles:
    all_filesets["WH"]["files"][f] = "Events"

In [4]:
# Look at ProcessorABC to see the expected methods and what they are supposed to do
class MyProcessor(processor.ProcessorABC):
    def __init__(self, isMC=False):

        ################################
        # INITIALIZE COFFEA PROCESSOR
        ################################ 

        self.isMC = isMC

        self.make_output = lambda: { 
            "Wkin": hda.Hist(
                hist.axis.Regular(20, 0, 1, name="pt_w1", label=r"W1 $p_T$ [GeV]"),
                hist.axis.Regular(20, 0, 1, name="pt_w2", label=r"W2 $p_T$ [GeV]"),
                hist.axis.Regular(20, 0, 1, name="eta_w1", label=r"W1 $\eta$"),
                hist.axis.Regular(20, 0, 1, name="eta_w2", label=r"W2 $\eta$"),
                storage=hist.storage.Weight()
            ),
            #"CosTheta": hist.Hist(
            #    hist.axis.Regular(20, 0, 1, name="JetCosTheta", label=r"$\cos\theta$ reco"),
            #    hist.axis.Regular(20, 0, 1, name="genJetCosTheta", label=r"$\cos\theta$ gen jet"),
            #    hist.axis.Regular(20, 0, 1, name="genPartCosTheta", label=r"$\cos\theta$ gen part"),
            #    storage=hist.storage.Weight()
            #    storage=hist.storage.Weight()
            #),
            "pTheta": hda.Hist(
                #hist.axis.Regular(20, 0, 1, name="pTheta", label=r"$p_\theta$ gen jet"),
                #hist.axis.Regular(20, 0, 1, name="genJetpTheta", label=r"$p_\theta$ gen jet"),
                hist.axis.Regular(20, 0, 1, name="genPartpTheta", label=r"$p_\theta$ gen jet"),
                hist.axis.Regular(20, 0, 1, name="genPartCosTheta", label=r"$\cos\theta$ gen part"),
                storage=hist.storage.Weight()
            ),
            "qTheta": hist.Hist(
                #hist.axis.Regular(20, 0, 1, name="qTheta", label=r"$q_\theta$ reco"),
                #hist.axis.Regular(20, 0, 1, name="genJetqTheta", label=r"$q_\theta$ gen jet"),
                hist.axis.Regular(20, 0, 1, name="genPartqTheta", label=r"$q_\theta$ gen part"),
                hist.axis.Regular(20, 0, 1, name="genPartCosTheta", label=r"$\cos\theta$ gen part"),
                storage=hist.storage.Weight()
            ),
            
            #"EventCount": processor.value_accumulator(int),
        }

    def process(self, events):
        
        output = self.make_output()

        ##################
        # OBJECT SELECTION
        ##################

        wbosons = events.GenPart[(abs(events.GenPart.pdgId)==24)
                           & (events.GenPart.hasFlags(["fromHardProcess","isLastCopy"]))
                          ]

        wbosons = wbosons[(wbosons.pt > 450) & (abs(wbosons.eta) < 2.5)]

        w1 = wbosons[:,0]
        w2 = wbosons[:,1]
 
        #####################
        # EVENT SELECTION
        #####################

        # create a PackedSelection object
        # this will help us later in composing the boolean selections easily
        #selection = PackedSelection()

        # Nothing for now, everything from generator

        ################
        # EVENT WEIGHTS
        ################

        # create a processor Weights object, with the same length as the number of events in the chunk
        #weights = Weights(len(events), storeIndividual=True)
        #gen_weights = np.ones(len(events)) #events.genWeight.to_numpy()
        #weights.add("genweight", gen_weights)

        # Why doesn't this work :(
        
        ###################
        # FILL HISTOGRAMS
        ###################

        #matchedJet = w.nearest(events.FatJet, threshold=0.8)
        #matchedJet["p4sum"] = matchedJet.subjets.sum(axis=2)

        #qqCM = w.children.boost(-w.boostvec)
        #subjetCM = matchedJet.subjets.boost(-matchedJet.p4sum.boostvec)
        #gencosTheta = ak.firsts((qqCM.pz / qqCM.rho)[:, :, 0])
        #cosTheta = ak.firsts((subjetCM.pz/subjetCM.rho)[:, :, 0])

        #subjetE = matchedJet.subjets.E

        #deltaE = abs(subjetE[:, :, 0] - subjetE[:, :, 1])
        #pV = matchedJet.p

        #pTheta = ak.firsts(deltaE/pV)

        #subjet_theta = matchedJet.subjets.theta

        #theta_op = abs(subjet_theta[:, :, 0] - subjet_theta[:, :, 1])
        #mW = genV.mass
        #EW = matchedJet.E

        #arg = mW/(EW * theta_op)

        #qTheta = ak.firsts(np.sqrt(1 - 4*arg*arg ))

        output['Wkin'].fill(pt_w1 = w1.pt,
                            pt_w2 = w2.pt,
                            eta_w1 = w1.eta,
                            eta_w2 = w2.eta,
                            weight=events.genWeight
                          )
        ##output['pTheta'].fill(genPartpTheta=genPartpTheta,
         #                     genPartCosTheta=genPartCosTheta,
         #                     weight=weights.weight()
         #                    )
        #output['qTheta'].fill(genPartqTheta=genPartqTheta,
        #                      genPartCosTheta=genPartCosTheta,
        #                      weight=weights.weight()
        #                     )
    
        #output["EventCount"] = ak.num(events, axis=0)
    
        return output

    def postprocess(self, accumulator):
        return accumulator


In [5]:
def run_samp(sample):

    fileset = {}
    fileset[sample] = all_filesets[sample]

    outfile = "coffea/"+sample+".coffea"

    print("Start preprocessing for " + sample)
    dataset_runnable, dataset_updated = preprocess(
        fileset[sample],
        align_clusters=False,
        step_size=100_000,
        files_per_batch=1,
        skip_bad_files=True,
        save_form=False,
    )

    print("Start processing for " + sample)
    to_compute = apply_to_fileset(
                MyProcessor(isMC=True),
                max_chunks(dataset_runnable, 300),
                schemaclass=NanoAODSchema,
            )

    print("Computing")
    (computed,) = dask.compute(to_compute)

    util.save(computed, outfile)
    print("saved " + outfile)

In [ ]:
run_samp("WW")

Start preprocessing for WW
